Draft conversion of ORCHID_COVID19.ipynb from R to python.

# ORCHID Clinical Trial: statistical analysis reproduction

# Version 1.0

This notebook reproduces the statistical analysis of the ORCHID clinical trial. Results have been published to JAMA, on November 7th 2021: ["Effect of Hydroxychloroquine on Clinical Status at 14 Days in Hospitalized Patients With COVID-19"](https://jamanetwork.com/journals/jama/fullarticle/2772922). The statistical analysis plan can be found on [clinicaltrials.gov](https://clinicaltrials.gov/ct2/show/NCT04332991?term=orchid&cond=Covid19&cntry=US&draw=2&rank=1). 

The clinical trial has been conducted between April and July 2020, and stopped before enrollment completion for futility, finding no difference of efficacy between hydroxychloroquine and placebo. This notebook is a reproduction of the clinical trial results based on the clinical trial protocol and the investigators original source code.

# Data Access using PIC-SURE API

User access authentication works through a security token, which is passed to the API using the token.txt file (file to be created by the user). In order to know how to get your security token, please see [the README of the PIC-SURE API GitHub repo](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst).

# ORCHID Clinical Trial

It is a multi-center, double blinded, randomized clinical trial conducted to assess the efficacy of hydroxychloroquine in the treatment of COVID-19 in hospitalized patients.

NHLBI made available the data to every authorized investigators. Hence, this notebook enables anybody with authorized credentials to reproduce the ORCHID clinical trial results by showing how to:
1. Access the data using the PIC-SURE API
2. Reproduce the results of this study using the open-source R programming languages

# Overview of the statistical analysis plan

The primary outcome is the COVID-19 Outcome Scale (COS) assess at 14 days. The scale comprises 7 levels, as follows:
- 1, Dead
- 2, Hospitalized on invasive mechanical ventilation or ECMO
- 3, Hospitalized on non-invasive ventilation or high flow nasal cannula
- 4, Hospitalized on supplemental oxygen
- 5, Hospitalized not on supplemental oxygen
- 6, Not hospitalized with limitation in activity (continued symptoms)
- 7, Not hospitalized without limitation in activity (no symptoms)

This scale will also be assessed at different timepoints as secondary outcomes: day-3, day-7, and day-28.

This scale will be treated as an ordered factor, and thus this outcome will be analyzed using a proportional odds regression model. 

Other secondary outcomes will be considered. Death, and composite of death and ECMO will be analyzed using logistic regression models. Time to recovery (patient without oxygen supplementation -- ie COS level 6 or 7) and time to discharge will be analyzed using survival models. Support free-days (hospital, oxygen, ICU, ventilation, and vasopressor) will be treated as ordered factors and analyzed using proportional odds regression models.

All these outcomes will be analyzed using multivariable models, taking into account the following potential confoundings: 
- Age at randomization
- Sex
- Clinical status as assessed by the COVID Ordinal Outcome Scale at randomization
- Sequential Organ Failure Assessment (SOFA) score at randomization
- Duration of acute respiratory infection symptoms prior to randomization

# Packages Installation 

In [2]:
# Install packages needed to use PIC-SURE
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git

  Cloning https://github.com/hms-dbmi/pic-sure-python-client.git to /tmp/pip-req-build-8jgmad8l
  Running command git clone -q https://github.com/hms-dbmi/pic-sure-python-client.git /tmp/pip-req-build-8jgmad8l
  Created wheel for PicSureClient: filename=PicSureClient-0.1.0-py2.py3-none-any.whl size=10225 sha256=fee93ed91d838e10202c886999c6817f80f3610707d3141f58ad3e98e725ef17
  Stored in directory: /tmp/pip-ephem-wheel-cache-e16j8vsr/wheels/31/ef/21/e362bba8de04e0072fafec9f77bd1abdf7e166213d27e98729
Successfully built PicSureClient
  Cloning https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git to /tmp/pip-req-build-6coizxbo
  Running command git clone -q https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git /tmp/pip-req-build-6coizxbo
     |████████████████████████████████| 95 kB 6.3 MB/s  eta 0:00:01
     |████████████████████████████████| 67 kB 1.2 MB/s  eta 0:00:01
  Created wheel for PicSureHpdsLib: filename=PicSureHpdsLib-0.9.0-py2.py3-none-any.whl size=21999 sha256=

In [3]:
!cat requirements.txt

numpy==1.16.4
matplotlib>=3.1.1
pandas>=0.25.3
scipy>=1.3.1
tqdm>=4.38.0
statsmodels>=0.10.2


In [5]:
!{sys.executable} -m pip install -r requirements.txt

     |████████████████████████████████| 17.3 MB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 7.4 MB/s  eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyarrow 3.0.0 requires numpy>=1.16.6, but you have numpy 1.16.4 which is incompatible.


In [6]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureClient
import PicSureBdcAdapter

from python_lib.utils import get_multiIndex_variablesDict, joining_variablesDict_onCol

# Installing the library and connecting to the database using the PICSURE API 

In [7]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"